### Converting Argo data to parquet with multiprocessing

**NB**: this solution will probably be replaced by the dask approach shown in `to_parquet_dask.ipynb`.

This notebook downloads and converts Argo Core and BGC profiles, given:

* the local path `gdac_path` to the argo index files (if they don't exist, they'll be downloaded to the folder),
* the path `outdir_nc` where to download the most recent Argo profile files (this is required to end with `GDAC/dac/`,
* the path `outdir_pqt` where the parquet database will be stored

In [1]:
import argo_tools as at
from pprint import pprint

gdac_path = '/vortexfs1/share/boom/data/nc2pqt_test/'
outdir_nc = '/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/'
outdir_pqt = '/vortexfs1/share/boom/data/nc2pqt_test/pqt/data/'

#### Downloading Argo profiles and generating list of file paths

The following cell downloads the most recent version of the profiles from the GDAC, and returns the list of paths to each stacked profile file (`*_prof.nc` and `*_Sprof.nc` files, for Core and BGC Argo respectively). For the Argo Core database, set `dataset='phy'`, for the BGC database, set `dataset='bgc'`.

If you already have the profiles stored somewhere, you can set the arguments `skip_downloads=False` and `dryrun=True` to simply generate the path list without downloading the profiles (or you can generate the list of file pahts yourself, just call it `flistPHY` or `flistBGC` and the rest of the notebook should work).

If you don't want the download to be multithreaded, set the argument `NPROC=1`.

##### BGC dataset

In [2]:
%%time
# bgc
wmos, df2, wmos_fp = at.argo_gdac(gdac_path=gdac_path, dataset='bgc', save_to=outdir_nc, download_individual_profs=False, skip_downloads=False, dryrun=False, overwrite_profiles=True, NPROC=20, verbose=True, checktime=True)

/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/src/argo_tools.py:92: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  gdac_index = pd.read_csv(gdac_file,delimiter=',',header=8,parse_dates=['date','date_update'],


More processors than files requested, limiting NPROC to number of files.
nb_to_download: 5
#0: >>> (0%) File 0 of 1...
#1: >>> (0%) File 0 of 1...#2: >>> (0%) File 0 of 1...#4: >>> (0%) File 0 of 1...#3: >>> (0%) File 0 of 1...#0: >>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902104/3902104_Sprof.nc.




#1: >>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902115/3902115_Sprof.nc.#4: >>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/csio/2902836/2902836_Sprof.nc.#2: >>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/7900586/7900586_Sprof.nc.#3: >>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/7900587/7900587_Sprof.nc.



#4: >>> File 2902836_Sprof.nc returned 404 error during download (requested URL: https://usgodae.org/pub/outgoing/argo/dac/csio/2902836/2902836_Sprof.nc).
#0: >>> File 3902104_Sprof.nc returned 404 error during download (re

#### File conversion

The conversion from netCDF to parquet uses multiprocessing to parallelize the operations of loading into memory multiple datasets and convert them taking in consideration their in-memory size.

The new parquet files will be stored in the directory `outdir_pqt` that you specified earlier.

In the next cell we set the parameters needed for the conversions (e.g. the database name) and then execute the conversion.

In [3]:
from convertTools import convertTools

converterBGC = convertTools(
    db_type="BGC",
    out_dir=outdir_pqt,
    flist=wmos_fp,
    metadata_table = df2,
    single_process = False
)

In [4]:
converterBGC.convert()

/vortexfs1/share/boom/data/nc2pqt_test/pqt/data/metadata/test_metadata.parquet stored.
Processing 2252 files.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902104/3902104_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/3902104/3902104_Sprof.nc. Skipping it.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902115/3902115_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/3902115/3902115_Sprof.nc. Skipping it.


...

File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/7900587/7900587_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/coriolis/7900587/7900587_Sprof.nc. Skipping it.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/csio/2902836/2902836_Sprof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/csio/2902836/2902836_Sprof.nc. Skipping it.

Size per processor (MB)
39.95466166398058

99.08544540405273
97.73884105682373
96.52665042877197
89.80551052093506
87.98799419403076
87.86990928649902
87.43379592895508
85.42619132995605
85.22486114501953
83.15616035461426
82.52131366729736
81.72994232177734
81.44808578491211
80.78289413452148
79.62730979919434
76.59406471252441
72.35435104370117
69.35292625427246
66.3397216796875
65.59677600860596
63.51381015777588
61.91689109802246
57.574278831481934
57.280272483825684
56.58053207397461
54.36483383178711
52.90372276306152
52.17085838317871
51.750932693481445
50.643943786621094
50.5104455947876
48.93765640258789
48.876712799072266
48.60991191864014
48.37381649017334
47.92483425140381
47.206130027770996
46.32355880737305
45.93830585479736
45.288790702819824
44.82143306732178
44.7547426223754

/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'argo' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


#11: Failed on /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/5906297/5906297_Sprof.nc
#9: Storing to parquet...
#9: In-memory filesize: 94.19 MB
#9: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_9_2.parquet stored.
#2: Storing to parquet...
#2: In-memory filesize: 235.25 MB
#13: Storing to parquet...
#13: In-memory filesize: 139.45 MB
#4: Storing to parquet...
#4: In-memory filesize: 206.55 MB
#13: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_13_2.parquet stored.
#2: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_2_2.parquet stored.
#18: Storing to parquet...
#18: In-memory filesize: 53.31 MB
#15: Storing to parquet...
#15: In-memory filesize: 99.81 MB
#4: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_4_2.parquet stored.
#18: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_18_2.parquet stored.
#15: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_1

##### PHY dataset

The same steps we used for the BGC dataset are here reproduced for the Core Argo data too.

In [6]:
%%time
# phy
wmosPHY, df2PHY, wmos_fpPHY = at.argo_gdac(gdac_path=gdac_path, dataset='phy', save_to=outdir_nc, download_individual_profs=False, skip_downloads=False, dryrun=True, overwrite_profiles=True, NPROC=10, verbose=True, checktime=True)

/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/src/argo_tools.py:92: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  gdac_index = pd.read_csv(gdac_file,delimiter=',',header=8,parse_dates=['date','date_update'],


CPU times: user 2min 27s, sys: 4 s, total: 2min 31s
Wall time: 3min 48s


Percentage of files whose conversion failed:

In [13]:
len(converterBGC.failed)/len(converterBGC.flist)*100

0.7104795737122558

In [7]:
from convertTools import convertTools

converterPHY = convertTools(
    db_type="PHY",
    out_dir=outdir_pqt,
    flist=wmos_fpPHY,
    metadata_table = df2PHY,
    single_process = False
)

In [8]:
converterPHY.convert()

/vortexfs1/share/boom/data/nc2pqt_test/pqt/data/metadata/test_metadata.parquet stored.
Processing 18514 files.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/bodc/1902116/1902116_prof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/bodc/1902116/1902116_prof.nc. Skipping it.


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'usgodae.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/bodc/1902595/1902595_prof.nc returned 404 error from URL https://usgodae.org/pub/outgoing/argo/dac/bodc/1902595/1902595_prof.nc. Skipping it.


...

#1: Failed on /vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/jma/2900741/2900741_prof.nc
#1: Storing to parquet...
#1: In-memory filesize: 99.96 MB
#1: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_1_76.parquet stored.
#2: processing file 20 of 29


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)
/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)


#3: processing file 10 of 15


/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/venv/venv3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)


#2: Storing to parquet...
#2: In-memory filesize: 74.82 MB
#2: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_2_76.parquet stored.
#3: Storing to parquet...
#3: In-memory filesize: 27.05 MB
#3: /vortexfs1/share/boom/data/nc2pqt_test/pqt/data/test_profiles_levels_3_76.parquet stored.
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/coriolis/3902117/3902117_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/5906556/5906556_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/1900061/1900061_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/7900688/7900688_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/7900045/7900045_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/5905986/5905986_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/5906154/5906154_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/aoml/5906563/5906563_prof.nc
/vortexfs1/share/boom/data/nc2pqt_test/GDAC/dac/incois/2901335/2901335_pro

Percentage of files whose conversion failed:

In [12]:
len(converterPHY.failed)/len(converterPHY.flist)*100

0.6913686939613265